In [1]:
%load_ext Cython

In [2]:
import array
import numpy as np

In [3]:
%%cython --annotate

import binascii
from math import ceil
import array

# rotl = lambda x, n:((x << n) & 0xffffffff) | ((x >> (32 - n)) & 0xffffffff)
bytes_to_list = lambda data: [i for i in data]


cdef unsigned int[:] IV = array.array('I', [
    1937774191, 1226093241, 388252375, 3666478592,
    2842636476, 372324522, 3817729613, 2969243214,
])

cdef unsigned int[:] T_j = array.array('I', [
    2043430169, 2043430169, 2043430169, 2043430169, 2043430169, 2043430169,
    2043430169, 2043430169, 2043430169, 2043430169, 2043430169, 2043430169,
    2043430169, 2043430169, 2043430169, 2043430169, 2055708042, 2055708042,
    2055708042, 2055708042, 2055708042, 2055708042, 2055708042, 2055708042,
    2055708042, 2055708042, 2055708042, 2055708042, 2055708042, 2055708042,
    2055708042, 2055708042, 2055708042, 2055708042, 2055708042, 2055708042,
    2055708042, 2055708042, 2055708042, 2055708042, 2055708042, 2055708042,
    2055708042, 2055708042, 2055708042, 2055708042, 2055708042, 2055708042,
    2055708042, 2055708042, 2055708042, 2055708042, 2055708042, 2055708042,
    2055708042, 2055708042, 2055708042, 2055708042, 2055708042, 2055708042,
    2055708042, 2055708042, 2055708042, 2055708042
])

cdef unsigned int rotl(unsigned int x, unsigned int n):
    return ((x << n)) | ((x >> (32 - n)))

cdef unsigned int sm3_ff_j(unsigned int x, unsigned int y, 
                           unsigned int z, unsigned int j):
    if 0 <= j and j < 16:
        ret = x ^ y ^ z
    elif 16 <= j and j < 64:
        ret = (x & y) | (x & z) | (y & z)
    return ret

cdef unsigned int sm3_gg_j(unsigned int x, unsigned int y, 
                          unsigned int z, unsigned int j):
    if 0 <= j and j < 16:
        ret = x ^ y ^ z
    elif 16 <= j and j < 64:
        #ret = (X | Y) & ((2 ** 32 - 1 - X) | Z)
        ret = (x & y) | ((~ x) & z)
    return ret

cdef unsigned int sm3_p_0(unsigned int x):
    return x ^ (rotl(x, 9 % 32)) ^ (rotl(x, 17 % 32))

cdef unsigned int sm3_p_1(unsigned int x):
    return x ^ (rotl(x, 15 % 32)) ^ (rotl(x, 23 % 32))

cdef void sm3_cf(unsigned int[:] v_j, unsigned int[:] v_i, unsigned int[:] b_i):
    cdef unsigned int w[68]
    cdef int weight, data
    cdef unsigned long i, j
    
    for i in range(68):
        w[i] = 0
    
    for i in range(16):
        weight = 0x1000000
        data = 0
        for k in range(i*4,(i+1)*4):
            data = data + b_i[k]*weight
            weight = weight//0x100
        w[i] = data

    for j in range(16, 68):
        w[j] = sm3_p_1(w[j-16] ^ w[j-9] ^ (rotl(w[j-3], 15 % 32))) ^ (rotl(w[j-13], 7 % 32)) ^ w[j-6]
#         str1 = "%08x" % w[j]
    
    cdef unsigned int w_1[64]
    
    for i in range(64):
        w_1[i] = 0
    
    for j in range(0, 64):
        w_1[j] = w[j] ^ w[j+4]
#         str1 = "%08x" % w_1[j]

#     a, b, c, d, e, f, g, h = v_i
    cdef unsigned int a, b, c, d, e, f, g, h
    a = v_i[0]
    b = v_i[1]
    c = v_i[2]
    d = v_i[3]
    e = v_i[4]
    f = v_i[5]
    g = v_i[6]
    h = v_i[7]
    
    cdef unsigned int ss_1, ss_2, tt_1, tt_2

    for j in range(0, 64):
        ss_1 = rotl(
            ((rotl(a, 12 % 32)) +
            e +
            (rotl(T_j[j], j % 32))),# & 0xffffffff, 
            7 % 32
        )
        ss_2 = ss_1 ^ (rotl(a, 12 % 32))
        tt_1 = (sm3_ff_j(a, b, c, j) + d + ss_2 + w_1[j])# & 0xffffffff
        tt_2 = (sm3_gg_j(e, f, g, j) + h + ss_1 + w[j])# & 0xffffffff
        d = c
        c = rotl(b, 9 % 32)
        b = a
        a = tt_1
        h = g
        g = rotl(f, 19 % 32)
        f = e
        e = sm3_p_0(tt_2)

#         a, b, c, d, e, f, g, h = map(
#             lambda x:x & 0xFFFFFFFF ,[a, b, c, d, e, f, g, h])
#         a = a & 0xFFFFFFFF
#         b = b & 0xFFFFFFFF
#         c = c & 0xFFFFFFFF
#         d = d & 0xFFFFFFFF
#         e = e & 0xFFFFFFFF
#         f = f & 0xFFFFFFFF
#         g = g & 0xFFFFFFFF
#         h = h & 0xFFFFFFFF
    
    v_j[0] = a ^ v_i[0]
    v_j[1] = b ^ v_i[1]
    v_j[2] = c ^ v_i[2]
    v_j[3] = d ^ v_i[3]
    v_j[4] = e ^ v_i[4]
    v_j[5] = f ^ v_i[5]
    v_j[6] = g ^ v_i[6]
    v_j[7] = h ^ v_i[7]

def sm3_hash(msg):
    msg = list(msg)
    len1 = len(msg)
    reserve1 = len1 % 64
    msg.append(0x80)
    reserve1 = reserve1 + 1
    # 56-64, add 64 byte
    range_end = 56
    if reserve1 > range_end:
        range_end = range_end + 64

    for i in range(reserve1, range_end):
        msg.append(0x00)

    bit_length = (len1) * 8
    bit_length_str = [bit_length % 0x100]
    for i in range(7):
        bit_length = int(bit_length / 0x100)
        bit_length_str.append(bit_length % 0x100)
    for i in range(8):
        msg.append(bit_length_str[7-i])

    group_count = round(len(msg) / 64)

    B = []
    for i in range(0, group_count):
        B.append(array.array('I', msg[i*64:(i+1)*64]))

    V = []
    V.append(IV)
    
    for i in range(0, group_count):
#         print(V[i], B[i])
        buf = array.array('I', [0] * 8)
        sm3_cf(buf, V[i], B[i])
        V.append(buf)

    y = V[i+1]
    result = ""
    for i in y:
        result = '%s%08x' % (result, i)
    return result

def sm3_kdf(z, klen): # z为16进制表示的比特串（str），klen为密钥长度（单位byte）
    klen = int(klen)
    ct = 0x00000001
    rcnt = ceil(klen/32)
    zin = bytes.fromhex(z.decode('utf8'))
    
    ha = ""
    for i in range(rcnt):
        msg = zin  + bytes.fromhex('%08x' % ct)
        ha = ha + sm3_hash(msg)
        ct += 1
    return ha[0: klen * 2]

In [14]:
import binascii
from random import choice
import os


# 选择素域，设置椭圆曲线参数

N = int('FFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFF7203DF6B21C6052B53BBF40939D54123', base=16)
P = int('FFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF00000000FFFFFFFFFFFFFFFF', base=16)
G = (
    int('32c4ae2c1f1981195f9904466a39c9948fe30bbff2660be1715a4589334c74c7', base=16),
    int('bc3736a2f4f6779c59bdcee36b692153d0a9877cc62a474002df32e52139f0a0', base=16),
    1)

A = int('FFFFFFFEFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFFF00000000FFFFFFFFFFFFFFFC', base=16)
B = int('28E9FA9E9D9F5E344D5A9E4BCF6509A7F39789F515AB8F92DDBCBD414D940E93', base=16)

ECC_A3 = (A + 3) % P

def point2hex(Point):
    x, y, z = Point
    
    if z != 1:
        return '%064x%064x%064x' % (x, y, z)
    else:
        return '%064x%064x' % (x, y)
    
def hex2point(s):
    if len(s) == 128:
        return (
            int(s[:64], base=16),
            int(s[64:], base=16),
            1
        )
    else:
        return (
            int(s[   :  64], base=16),
            int(s[64 : 128], base=16),
            int(s[128: 192], base=16)
        )

def kP(k, Point):  # kP运算
    mask = int('8' + '0' * 63, 16)
    Temp = Point
    flag = False
    for n in range(64 * 4):
        if flag:
            Temp = double_point(Temp)
        if (k & mask) != 0:
            if flag:
                Temp = add_point(Temp, Point)
            else:
                flag = True
                Temp = Point
        k = k << 1
    return convert_jacb_to_nor(Temp)


def double_point(Point):  # 倍点
    """
    Point: (x, y, z)
    
    return double_point
    """
    x1, y1, z1 = Point

    T6 = (z1 * z1) % P
    T2 = (y1 * y1) % P
    T3 = (x1 + T6) % P
    T4 = (x1 - T6) % P
    T1 = (T3 * T4) % P
    T3 = (y1 * z1) % P
    T4 = (T2 * 8) % P
    T5 = (x1 * T4) % P
    T1 = (T1 * 3) % P
    T6 = (T6 * T6) % P
    T6 = (ECC_A3 * T6) % P
    T1 = (T1 + T6) % P
    z3 = (T3 + T3) % P
    T3 = (T1 * T1) % P
    T2 = (T2 * T4) % P
    x3 = (T3 - T5) % P

    if (T5 % 2) == 1:
        T4 = (T5 + ((T5 + P) >> 1) - T3) % P
    else:
        T4 = (T5 + (T5 >> 1) - T3) % P

    T1 = (T1 * T4) % P
    y3 = (T1 - T2) % P

    return x3, y3, z3

def add_point(P1, P2):  # 点加函数，P2点为仿射坐标即z=1，P1为Jacobian加重射影坐标
    """
    P1: (x, y, z)
    P1: (x, y, z)
    
    return add_point
    """
    X1, Y1, Z1 = P1
    x2, y2, z2 = P2

    T1 = (Z1 * Z1) % P
    T2 = (y2 * Z1) % P
    T3 = (x2 * T1) % P
    T1 = (T1 * T2) % P
    T2 = (T3 - X1) % P
    T3 = (T3 + X1) % P
    T4 = (T2 * T2) % P
    T1 = (T1 - Y1) % P
    Z3 = (Z1 * T2) % P
    T2 = (T2 * T4) % P
    T3 = (T3 * T4) % P
    T5 = (T1 * T1) % P
    T4 = (X1 * T4) % P
    X3 = (T5 - T3) % P
    T2 = (Y1 * T2) % P
    T3 = (T4 - X3) % P
    T1 = (T1 * T3) % P
    Y3 = (T1 - T2) % P

    return X3, Y3, Z3

def convert_jacb_to_nor(Point):  # Jacobian加重射影坐标转换成仿射坐标
    """
    Point: (x, y, z)
    
    return double_point
    """
    x, y, z = Point
    z_inv = pow(z, P - 2, P)
    z_invSquar = (z_inv * z_inv) % P
    z_invQube = (z_invSquar * z_inv) % P
    x_new = (x * z_invSquar) % P
    y_new = (y * z_invQube) % P
    z_new = (z * z_inv) % P
    if z_new == 1:
        return x_new, y_new, z_new
    else:
        return None
    

def generate_key():
    """
    return: sk, pk
    """
    
    k = os.urandom(32)
    k = int(k.hex(), 16)

    pk = kP(k, G)
    
    return ('%064x'% k).upper(), point2hex(pk).upper()
    

def encrypt(pk, k, data, mode='C1C3C2'):
    # 加密函数，data消息(bytes)
    msg = data.hex()  # 消息转化为16进制字符串
    k = int(k.hex(), 16)
    pk = hex2point(pk)
    
    C1 = kP(k, G)
    xy = kP(k, pk)
    
    x2, y2, _ = xy
    x2 = '%064x' % x2
    y2 = '%064x' % y2
    
    ml = len(msg)
    t = sm3_kdf(point2hex(xy).encode('utf8'), ml/2)
    if int(t, 16) == 0:
        return None
    else:
        form = '%%0%dx' % ml
        
        C1 = point2hex(C1)
        C2 = form % (int(msg, 16) ^ int(t, 16))
        C3 = sm3_hash(bytes.fromhex('%s%s%s' % (x2, msg, y2)))
        
        if mode == 'C1C3C2':
            return bytes.fromhex('%s%s%s' % (C1, C3, C2))
        elif mode == 'C1C2C3':
            return bytes.fromhex('%s%s%s' % (C1, C2, C3))
        else:
            return None

def decrypt(sk, data, mode='C1C3C2'):
    # 解密函数，data密文（bytes）
    data = data.hex()
    len_2 = 128
    len_3 = len_2 + 64
    C1 = data[0:len_2]

    if mode == 'C1C3C2':
        C3 = data[len_2:len_3]
        C2 = data[len_3:]
    elif mode == 'C1C2C3':
        C2 = data[len_2:-64]
        C3 = data[-64:]
    else:
        return None
        
    sk = int(sk, 16)
    C1 = hex2point(C1)
    
    xy = kP(sk, C1)
    x2, y2, _ = xy
    x2 = '%064x' % x2
    y2 = '%064x' % y2

    cl = len(C2)
    t = sm3_kdf(point2hex(xy).encode('utf8'), cl/2)
    if int(t, 16) == 0:
        return None
    else:
        form = '%%0%dx' % cl
        M = form % (int(C2, 16) ^ int(t, 16))
        u = sm3_hash(bytes.fromhex('%s%s%s' % (x2, M, y2)))
        
        if u == C3:
            return bytes.fromhex(M)
        else:
            return None

In [6]:
# %%timeit

sk = '00B9AB0B828FF68872F21A837FC303668428DEA11DCD1B24429D0C99E24EED83D5'
pk = 'B9C9A6E04E9C91F7BA880429273747D7EF5DDEB0BB2FF6317EB00BEF331A83081A6994B8993F3F5D6EADDDB81872266C87C018FB4162F5AF347B483E24620207'

k = os.urandom(32)
data = b'helloworld' * 1024
enc = encrypt(pk, k, data)
dec = decrypt(sk, enc)
enc, dec

(b'j\xd2d\xaf\xd7\x88)\xe6\x922\x84FP:saz\xb5\xfb\x89\xc8\xb8e!\x00\xc77l\x0eQ\xa4D\xc1\xffD1I\xe3\xf8\x18\xb5]o\xc9\xbf9-\xce\xe1\r\xaa!\x03\xf1!;5\xb9|\xc2\xee\x00X\xd2*\xec\xb5\x8f\xbc\xebKq<,\xca\x04\x8e\xbe\xb7\x98-,J\x8d\xda\x05\x81\xd4bE\xfdyqI48\xd3Z\xc4\xadD\xdf%\x03zc\x8d\xe7AJY\x95\xd6\x0cbz\xc6\xb2s@\x9d\xba\x17\x88\xa96\xeb\xd7}\r\xe0\xcclJ>\xb3\xbb\x84W\x95\xfd\xdat*jE^Vevt\xa9\x8dq\x92\xaa\xf1\x10\x05\xbc\xc1\x1c\x1c\xf6/\xb5\x04b\x8d\xfc\x88\x92:\x17\xe7\xd3S\nX\x10=T\xb7\xa8R(\x9b\x97\x9e"@\x8bk\xc9D\xd1\xc2\xddP|>\x9c<\xf2\xf0\rex}\xd7\xde\xd7\x11w\x13w\x1eM\x81\xe7\xcd\x02<\xcf\x9f\xf7#Bp\xfb\x1c\x98%\n\xce\xcb\xd3AD\x86q\x82\xa3\xe5C\xbaU\xabZ\x97\xb3\x04$\xb7\xbf\xdb\x8c\x13Qm\\E\xb7\xed\xc8\x88;\x89\x07\x1d0\xe2\xac\x9bakL\xcc\xa8\x15\xcdw\x19\x1d\xf1\xa5\xa49JL\x19\xb5z\xc1z\xcd\x81\xaas2\x85\xc1\x90\xab\x82\xa3\xb3\x88\t\xc7\xac`\xa9\x80(\x0b\xbe\x98\x95\x85\xb1\x00\xe8\xdf\xedK\xe2\xc5\x8e\xe0U\xa8\xc0\x14\xf4g\xde\xa1\x97\xa4[\xdeV!1\x1d\xb6\xf3\x1f2i\xe4\r1\x

In [17]:
sk, pk = generate_key()

k = os.urandom(32)
data = b'helloworld'
enc = encrypt(pk, k, data)
dec = decrypt(sk, enc)
enc, dec

(b'\x02\xce\xffIQ\xb5\x8e"j\x95\x1d@\xbe\x8f\xcc\x89\xfd\xbbdo\xb6<\xeb0\x0f\x1e\xf8%\xd0i\xc4{\xa4\x04lgR\xaa*\xfc4\xfe\xd1\x03\xc0@\x1b\xcb\x05\xe3\xf2\x00\xb2A\x0ez^\xa6\x16\xff\xf8\x96\x13}\x00\xcbM\xdevH\x81\xa7\xf1\x08\x9e\xc20\x04\x03m1_\xbaO?\xe7\x91\xbaz\xb6k\x1f\x9d$\x80\xdb\xff\xe7\x8c\x83\xbd\x8b\x1e\xfc\x93\x89',
 b'helloworld')

In [48]:
%%timeit

sk = '00B9AB0B828FF68872F21A837FC303668428DEA11DCD1B24429D0C99E24EED83D5'
pk = 'B9C9A6E04E9C91F7BA880429273747D7EF5DDEB0BB2FF6317EB00BEF331A83081A6994B8993F3F5D6EADDDB81872266C87C018FB4162F5AF347B483E24620207'

k = b'123'
data = b'helloworld' * 1024
enc = encrypt(pk, k, data)
dec = decrypt(sk, enc)
enc, dec

18.8 ms ± 314 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


In [22]:
%load_ext line_profiler

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


In [42]:
%lprun -f encrypt encrypt(pk, k, data)

In [103]:
from gmssl.sm2 import CryptSM2

In [119]:
%%timeit
sk = '00B9AB0B828FF68872F21A837FC303668428DEA11DCD1B24429D0C99E24EED83D5'
pk = 'B9C9A6E04E9C91F7BA880429273747D7EF5DDEB0BB2FF6317EB00BEF331A83081A6994B8993F3F5D6EADDDB81872266C87C018FB4162F5AF347B483E24620207'

data = b'helloworld' * 1024

sm2 = CryptSM2(public_key=pk, private_key=sk)
sm2.decrypt(sm2.encrypt(data))

974 ms ± 6.86 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [52]:
%%cython --annotate
import cython
from cpython cimport array
from libc.math cimport floor
import array
# import math

cdef class Point:
    cdef unsigned int[:] X
    cdef unsigned int[:] Y
    cdef unsigned int[:] Z
    
    def __init__(self, unsigned int[:] X, unsigned int[:] Y, unsigned int[:] Z):
        self.X = X
        self.Y = Y
        self.Z = Z

cdef unsigned int[:] SM2_P = array.array('I', [
    0xffff, 0xffff, 0xffff, 0xffff,
    0x0000, 0x0000, 0xffff, 0xffff,
    0xffff, 0xffff, 0xffff, 0xffff,
    0xffff, 0xffff, 0xfffe, 0xffff,
])

cdef unsigned int[:] SM2_B = array.array('I', [
    0x0e93, 0x4d94, 0xbd41, 0xddbc,
    0x8f92, 0x15ab, 0x89f5, 0xf397,
    0x09a7, 0xcf65, 0x9e4b, 0x4d5a,
    0x5e34, 0x9d9f, 0xfa9e, 0x28e9,
])


cdef SM2_G = Point(
    array.array('I', [
    0x74c7, 0x334c, 0x4589, 0x715a,
    0x0be1, 0xf266, 0x0bbf, 0x8fe3,
    0xc994, 0x6a39, 0x0446, 0x5f99,
    0x8119, 0x1f19, 0xae2c, 0x32c4]),
    
    array.array('I', [0xf0a0, 0x2139, 0x32e5, 0x02df,
    0x4740, 0xc62a, 0x877c, 0xd0a9,
    0x2153, 0x6b69, 0xcee3, 0x59bd,
    0x779c, 0xf4f6, 0x36a2, 0xbc37]),
    
    array.array('I', [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
)
# cdef (unsigned int[:], unsigned int[:], unsigned int[:]) SM2_G = (
#     array.array('I', [
#     0x74c7, 0x334c, 0x4589, 0x715a,
#     0x0be1, 0xf266, 0x0bbf, 0x8fe3,
#     0xc994, 0x6a39, 0x0446, 0x5f99,
#     0x8119, 0x1f19, 0xae2c, 0x32c4]),
    
#     array.array('I', [0xf0a0, 0x2139, 0x32e5, 0x02df,
#     0x4740, 0xc62a, 0x877c, 0xd0a9,
#     0x2153, 0x6b69, 0xcee3, 0x59bd,
#     0x779c, 0xf4f6, 0x36a2, 0xbc37]),
    
#     array.array('I', [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
# )

cdef unsigned int[:] SM2_N = array.array('I', [
    0x4123, 0x39d5, 0xf409, 0x53bb,
    0x052b, 0x21c6, 0xdf6b, 0x7203,
    0xffff, 0xffff,    0xffff,    0xffff,
    0xffff,    0xffff,    0xfffe,    0xffff,
])

def bn_new():
    return array.array('I', [0] * 16)

# def bn_free(a):
#     bn_set_zero(a)
#     delete a
#

cdef int bn_is_zero(unsigned int[:] a):
    cdef unsigned long i
    for i in range(16):
        if a[i] != 0:
            return 0
    return 1

cdef int bn_is_one(unsigned int[:] a):
    cdef unsigned int i
    if a[0] != 1:
        return 0
    for i in range(16):
        if a[i] != 0:
            return 0
    return 1

cdef void bn_to_bytes(unsigned int[:] a, unsigned char[:] buf, unsigned long offset):
    cdef unsigned int i
    for i in range(15, -1, -1):
        buf[offset] = a[i] // 256
        offset += 1
        buf[offset] = a[i] % 256
        offset += 1

cdef void bn_set_bytes(unsigned int[:] a, unsigned char[:] buf, unsigned long offset):
    cdef unsigned int i
    for i in range(15, -1, -1):
        a[i] = buf[offset] * 256
        offset += 1
        a[i] += buf[offset]
        offset += 1

cdef int bn_cmp(unsigned int[:] a, unsigned int[:] b):
    cdef unsigned int i
    for i in range(15, -1, -1):
        if a[i] > b[i]:
            return 1
        if a[i] < b[i]:
            return -1
    return 0

cdef void bn_set_word(unsigned int[:] a, unsigned int word):
    cdef unsigned int i
    a[0] = word
    for i in range(1, 16):
        a[i] = 0

cdef void bn_set_zero(unsigned int[:] a):
    bn_set_word(a, 0)

cdef void bn_set_one(unsigned int[:] a):
    bn_set_word(a, 1)

cdef void bn_copy(unsigned int[:] r, unsigned int[:] a):
    cdef unsigned int i
    for i in range(16):
        r[i] = a[i]

cdef bn_to_bits(unsigned int[:] a):
    cdef unsigned int i
#     cdef unsigned long w
    bits = [0] * 256
    for i in range(16):
        w = a[i]
        for j in range(16):
            bits[i*16 + j] = w % 2
            w = w//2
    return reversed(bits)

cdef void bn_add(unsigned int[:] r, unsigned int[:] a, unsigned int[:] b):
    cdef unsigned int i
    r[0] = a[0] + b[0]
    for i in range(1, 16):
        r[i] = a[i] + b[i] + r[i-1] // 65536
    for i in range(15):
        r[i] = r[i] % 65536

cdef void bn_sub(unsigned int[:] r, unsigned int[:] a, unsigned int[:] b):
    cdef unsigned int i
    cdef unsigned int borrow = 0
    for i in range(16):
        r[i] = a[i] - b[i] - borrow
        borrow = 0
        if r[i] < 0:
            r[i] += 65536
            borrow = 1

cdef void fp_add(unsigned int[:] r, unsigned int[:] a, unsigned int[:] b):
    bn_add(r, a, b)
    if bn_cmp(r, SM2_P) >= 0:
        bn_sub(r, r, SM2_P)

cdef void fp_sub(unsigned int[:] r, unsigned int[:] a, unsigned int[:] b):
    if bn_cmp(a, b) >= 0:
        bn_sub(r, a, b) 
    else:
        # FIXME:  remove t,
        # it need there is no fp_sub(b, a, b)
        # which is used in point_double()
        t = bn_new()
        bn_add(t, a, SM2_P)
        bn_sub(r, t, b)

cdef void fp_dbl(unsigned int[:] r, unsigned int[:] a):
    fp_add(r, a, a)

cdef void fp_tri(unsigned int[:] r, unsigned int[:] a):
    t = bn_new()
    fp_dbl(t, a)
    fp_add(r, t, a)

cdef void fp_div2(unsigned int[:] r, unsigned int[:] a):
    bn_copy(r, a)
    if r[0] % 2:
        bn_add(r, r, SM2_P)

    for i in range(15):
        r[i] = r[i] // 2
        r[i] += (r[i + 1] % 2) * 32768
    r[i] = r[i] // 2

cdef void fp_neg(unsigned int[:] r, unsigned int[:] a):
    if bn_is_zero(a):
        bn_set_zero(r) 
    else:
        bn_sub(r, SM2_P, a)

cdef void fp_mul(unsigned int[:] r, unsigned int[:] a, unsigned int[:] b):
    cdef unsigned int[:] ab = array.array('I', [0] * 32)

    for i in range(16):
        for j in range(16):
            ab[i + j] += a[i] * b[j]
    for i in range(31):
        ab[i + 1] += ab[i] // 65536
        ab[i] = ab[i] % 65536
    cdef unsigned int[:] s = ab[16: 32]

    r[ 0] = ab[ 0] + s[0] + s[ 2] + s[ 4] + s[ 6] + s[ 8] + 2 * (s[10] + s[12] + s[14])
    r[ 1] = ab[ 1] + s[1] + s[ 3] + s[ 5] + s[ 7] + s[ 9] + 2 * (s[11] + s[13] + s[15])
    r[ 2] = ab[ 2] + s[2] + s[ 4] + s[ 6] + s[ 8] + s[10] + 2 * (s[12] + s[14])
    r[ 3] = ab[ 3] + s[3] + s[ 5] + s[ 7] + s[ 9] + s[11] + 2 * (s[13] + s[15])
    r[ 4] = ab[ 4]
    r[ 5] = ab[ 5]
    r[ 6] = ab[ 6] + s[0] + s[ 6] + s[ 8] + s[12] + s[14] + 2 * s[10]
    r[ 7] = ab[ 7] + s[1] + s[ 7] + s[ 9] + s[13] + s[15] + 2 * s[11]
    r[ 8] = ab[ 8] + s[2] + s[ 8] + s[10] + s[14]         + 2 * s[12]
    r[ 9] = ab[ 9] + s[3] + s[ 9] + s[11] + s[15]         + 2 * s[13]
    r[10] = ab[10] + s[4] + s[10] + s[12]                 + 2 * s[14]
    r[11] = ab[11] + s[5] + s[11] + s[13]                 + 2 * s[15]
    r[12] = ab[12] + s[6] + s[12] + s[14]
    r[13] = ab[13] + s[7] + s[13] + s[15]
    r[14] = ab[14] + s[0] + s[ 2] + s[ 4] + s[ 6] + s[14] + 2 * (s[8] + s[10] + s[12] + s[14])
    r[15] = ab[15] + s[1] + s[ 3] + s[ 5] + s[ 7] + s[15] + 2 * (s[9] + s[11] + s[13] + s[15])

    for i in range(15):
        r[i + 1] += r[i] // 65536
        r[i] = r[i] % 65536

    d = [0,0,0,0,s[0]+s[2]+s[10]+s[12],s[1]+s[3]+s[11]+s[13],0,0,0,0,0,0,0,0,0,0]
    for i in range(15):
        d[i + 1] += d[i] // 65536
        d[i] = d[i] % 65536

    bn_sub(r, r, d)

    while bn_cmp(r, SM2_P) >= 0:
        bn_sub(r, r, SM2_P)

cdef void fp_sqr(unsigned int[:] r, unsigned int[:] a):
    fp_mul(r, a, a)

cdef void fp_inv(unsigned int[:] r, unsigned int[:] a):
    a1 = bn_new()
    a2 = bn_new()
    a3 = bn_new()
    a4 = bn_new()
    a5 = bn_new()

    fp_sqr(a1, a)
    fp_mul(a2, a1, a)
    fp_sqr(a3, a2)
    fp_sqr(a3, a3)
    fp_mul(a3, a3, a2)
    fp_sqr(a4, a3)
    fp_sqr(a4, a4)
    fp_sqr(a4, a4)
    fp_sqr(a4, a4)
    fp_mul(a4, a4, a3)
    fp_sqr(a5, a4)
    for i in range(1, 8):
        fp_sqr(a5, a5)
    fp_mul(a5, a5, a4)
    for i in range(1, 8):
        fp_sqr(a5, a5)
    fp_mul(a5, a5, a4)
    for i in range(1, 4):
        fp_sqr(a5, a5)
    fp_mul(a5, a5, a3)
    fp_sqr(a5, a5)
    fp_sqr(a5, a5)
    fp_mul(a5, a5, a2)
    fp_sqr(a5, a5)
    fp_mul(a5, a5, a)
    fp_sqr(a4, a5)
    fp_mul(a3, a4, a1)
    fp_sqr(a5, a4)
    for i in range(1, 31):
        fp_sqr(a5, a5)
    fp_mul(a4, a5, a4)
    fp_sqr(a4, a4)
    fp_mul(a4, a4, a)
    fp_mul(a3, a4, a2)
    for i in range(1, 33):
        fp_sqr(a5, a5)
    fp_mul(a2, a5, a3)
    fp_mul(a3, a2, a3)
    for i in range(1, 32):
        fp_sqr(a5, a5)
    fp_mul(a2, a5, a3)
    fp_mul(a3, a2, a3)
    fp_mul(a4, a2, a4)
    for i in range(1, 32):
        fp_sqr(a5, a5)
    fp_mul(a2, a5, a3)
    fp_mul(a3, a2, a3)
    fp_mul(a4, a2, a4)
    for i in range(1, 32):
        fp_sqr(a5, a5)
    fp_mul(a2, a5, a3)
    fp_mul(a3, a2, a3)
    fp_mul(a4, a2, a4)
    for i in range(1, 32):
        fp_sqr(a5, a5)
    fp_mul(a2, a5, a3)
    fp_mul(a3, a2, a3)
    fp_mul(a4, a2, a4)
    for i in range(1, 32):
        fp_sqr(a5, a5)
    fp_mul(r, a4, a5)

#     bn_free(a1)
#     bn_free(a2)
#     bn_free(a3)
#     bn_free(a4)
#     bn_free(a5)

cdef Point sm2_point_new():
    cdef unsigned int X = array.array('I', [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
    cdef unsigned int Y = array.array('I', [1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
    cdef unsigned int Z = array.array('I', [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0])
    
    return Point(X, Y, Z)

# # def sm2_point_free(P):
# #     bn_set_zero(P.X)
# #     bn_set_zero(P.Y)
# #     bn_set_zero(P[3])
# #     delete P

cdef void sm2_point_copy(Point R, Point P):
    bn_copy(R.X, P.X)
    bn_copy(R.Y, P.Y)
    bn_copy(R.Z, P.Z)

cdef int sm2_point_is_at_infinity(Point P):
    return bn_is_zero(P.Z)

cdef void sm2_point_set_infinity(Point R):
    bn_set_one(R.X)
    bn_set_one(R.Y)
    bn_set_zero(R.Z)

cdef void sm2_point_get_affine(Point R, Point P):
    sm2_point_copy(R, P)
    if (bn_is_zero(R.Z) or bn_is_one(R.Z)):
        return
    fp_inv(R.Z, R.Z)
    fp_mul(R.Y, R.Y, R.Z)
    fp_sqr(R.Z, R.Z)
    fp_mul(R.X, R.X, R.Z)
    fp_mul(R.Y, R.Y, R.Z)
    bn_set_one(R.Z)

cdef void sm2_point_set_bytes(Point P, unsigned char[:] buf, unsigned long offset):
    bn_set_bytes(P.X, buf, offset)
    bn_set_bytes(P.Y, buf, offset + 32)
    bn_set_one(P[3])

cdef void sm2_point_to_bytes(Point P, unsigned char[:] buf, unsigned long offset):
    A = sm2_point_new()
    sm2_point_get_affine(A, P)
    bn_to_bytes(A[0], buf, offset)
    bn_to_bytes(A[1], buf, offset + 32)

cdef int sm2_point_is_on_curve(Point P):
    t0 = bn_new()
    t1 = bn_new()
    t2 = bn_new()

    if bn_is_one(P[3]):
        fp_sqr(t0, P.Y)
        fp_add(t0, t0, P.X)
        fp_add(t0, t0, P.X)
        fp_add(t0, t0, P.X)
        fp_sqr(t1, P.X)
        fp_mul(t1, t1, P.X)
        fp_add(t1, t1, SM2_B) 
    else:
        fp_sqr(t0, P.Y)
        fp_sqr(t1, P[3])
        fp_sqr(t2, t1)
        fp_mul(t1, t1, t2)
        fp_mul(t1, t1, SM2_B)
        fp_mul(t2, t2, P.X)
        fp_add(t0, t0, t2)
        fp_add(t0, t0, t2)
        fp_add(t0, t0, t2)
        fp_sqr(t2, P.X)
        fp_mul(t2, t2, P.X)
        fp_add(t1, t1, t2)

    return (bn_cmp(t0, t1) == 0)

cdef void sm2_point_neg(Point R, Point P):
    bn_copy(R.X, P.X)
    fp_neg(R.Y, P.Y)
    bn_copy(R.Z, P.Z)

cdef void sm2_point_double(Point R, Point P):
    if sm2_point_is_at_infinity(P):
        sm2_point_copy(R, P)
        return 
    X1 = P.X
    Y1 = P.Y
    Z1 = P.Z
    T1 = bn_new()
    T2 = bn_new()
    T3 = bn_new()
    X3 = bn_new()
    Y3 = bn_new()
    Z3 = bn_new()

    fp_sqr(T1, Z1)
    fp_sub(T2, X1, T1)
    fp_add(T1, X1, T1)
    fp_mul(T2, T2, T1)
    fp_tri(T2, T2)
    fp_dbl(Y3, Y1)
    fp_mul(Z3, Y3, Z1)
    fp_sqr(Y3, Y3)
    fp_mul(T3, Y3, X1)
    fp_sqr(Y3, Y3)
    fp_div2(Y3, Y3)
    fp_sqr(X3, T2)
    fp_dbl(T1, T3)
    fp_sub(X3, X3, T1)
    fp_sub(T1, T3, X3)
    fp_mul(T1, T1, T2)
    fp_sub(Y3, T1, Y3)

    bn_copy(R.X, X3)
    bn_copy(R.Y, Y3)
    bn_copy(R.Z, Z3)

cdef void sm2_point_add(Point R, Point P, Point Q):
    if sm2_point_is_at_infinity(Q):
        sm2_point_copy(R, P)
        return
    if sm2_point_is_at_infinity(P):
        sm2_point_copy(R, Q)
        return

    X1 = P.X
    Y1 = P.Y
    Z1 = P[3]
    x2 = Q[0]
    y2 = Q[1]
    T1 = bn_new()
    T2 = bn_new()
    T3 = bn_new()
    T4 = bn_new()
    X3 = bn_new()
    Y3 = bn_new()
    Z3 = bn_new()

    fp_sqr(T1, Z1)
    fp_mul(T2, T1, Z1)
    fp_mul(T1, T1, x2)
    fp_mul(T2, T2, y2)
    fp_sub(T1, T1, X1)
    fp_sub(T2, T2, Y1)
    if bn_is_zero(T1):
        if bn_is_zero(T2):
            sm2_point_double(R, Q)
            return
        else:
            sm2_point_set_infinity(R)
            return
    fp_mul(Z3, Z1, T1)
    fp_sqr(T3, T1)
    fp_mul(T4, T3, T1)
    fp_mul(T3, T3, X1)
    fp_dbl(T1, T3)
    fp_sqr(X3, T2)
    fp_sub(X3, X3, T1)
    fp_sub(X3, X3, T4)
    fp_sub(T3, T3, X3)
    fp_mul(T3, T3, T2)
    fp_mul(T4, T4, Y1)
    fp_sub(Y3, T3, T4)

    bn_copy(R.X, X3)
    bn_copy(R.Y, Y3)
    bn_copy(R.Z, Z3)

cdef void sm2_point_mul(Point R, Point k, Point P):
    bits = bn_to_bits(k)
    Q = sm2_point_new()

    for i in range(len(bits)):
        sm2_point_double(Q, Q)

        if bits[i] == 1:
            sm2_point_add(Q, Q, P)
    sm2_point_copy(R, Q)

cdef void sm2_point_mul_G(Point R, Point k):
    sm2_point_mul(R, k, SM2_G)


# cdef void sm2_sig_new():
#     return (bn_new(), bn_new())

# cdef void sm2_do_verify(Point dgst, Point sig, Point P):

#     if (dgst.length < 32
#         or bn_is_zero(sig[0])
#         or bn_cmp(sig[0], SM2_N) >= 0
#         or bn_is_zero(sig[1])
#         or bn_cmp(sig[1], SM2_N) >= 0):
#         return False

#     e = bn_new()

#     bn_set_bytes(e, dgst, 0)
#     if bn_cmp(e, SM2_N) >= 0:
#         bn_sub(e, e, SM2_N)

#     t = bn_new()
#     bn_add(t, sig[0], sig[1])
#     if bn_cmp(t, SM2_N) == 0:
#         return False

#     Q1 = sm2_point_new()
#     Q2 = sm2_point_new()

#     sm2_point_mul_G(Q1, s1)
#     sm2_point_mul(Q2, t, P)
#     sm2_point_add(Q1, Q1, Q2)
#     sm2_point_get_affine(Q1, Q1)

#     bn_add(e, e, Q1[0])
#     if bn_cmp(e, SM2_N):
#         bn_sub(e, e, SM2_N)
#     if bn_cmp(e, SM2_N):
#         bn_sub(e, e, SM2_N)

#     return bn_cmp(e, sig[0]) == 0

In [ ]:


# def sm2_point_test():

#     P = sm2_point_new()

#     /*
#     sm2_point_double(P, SM2_G)
#     sm2_point_double(P, P)
#     sm2_point_add(P, P, SM2_G)
#     sm2_point_get_affine(P, P)
#     console.log(P)
#     */

#     k = bn_new()
#     bn_set_word(k, 11)

#     console.log("mul")
#     sm2_point_mul(P, SM2_B, SM2_G)
#     sm2_point_get_affine(P, P)

#     console.log(P[0])
#     console.log(P[1])
#     console.log(P[3])